In [19]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from matplotlib import image

from numpy import asarray

In [20]:
IMAGE_SIZE=256
BATCH_SIZE=32
TRAINING_SIZE=0.8
VALIDATION_SIZE=0.1
TEST_SIZE=0.1
CHANNEL=3
EPOCHS=50

In [21]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [22]:
class_names=dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [23]:
def split_data_to_Partion(dataset,TRAINING_SIZE,VALIDATIION_SIZE,TEST_SIZE):
    train_ds=dataset.take(int(len(dataset)*TRAINING_SIZE))
    validation_ds=dataset.skip(len(train_ds)).take(int(len(dataset)*VALIDATION_SIZE))
    teststartposition=len(train_ds)+len(validation_ds)
    test_ds=dataset.skip(teststartposition)

    return train_ds,validation_ds,test_ds

In [24]:
train_ds,validation_ds,test_ds=split_data_to_Partion(dataset,TRAINING_SIZE,VALIDATION_SIZE,TEST_SIZE)
print(len(train_ds),len(validation_ds),len(test_ds))

54 6 8


In [25]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [26]:
data_augmentation= tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [27]:
input_shape=(BATCH_SIZE,IMAGE_SIZE, IMAGE_SIZE,CHANNEL)
n=3
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape= input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n,activation='softmax'),
])
model.build(input_shape=input_shape)


In [28]:
model.compile(
optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
metrics=['accuracy']
)

In [ ]:
history= model.fit(
train_ds,
epochs=EPOCHS,
batch_size=BATCH_SIZE,
verbose=1,
validation_data=validation_ds
)

Epoch 1/50
54/54 [==============================] - 114s 2s/step - loss: 0.9037 - accuracy: 0.4630 - val_loss: 0.8441 - val_accuracy: 0.4844
Epoch 2/50
54/54 [==============================] - 107s 2s/step - loss: 0.6660 - accuracy: 0.6898 - val_loss: 0.4230 - val_accuracy: 0.8542
Epoch 3/50
54/54 [==============================] - 134s 2s/step - loss: 0.3475 - accuracy: 0.8536 - val_loss: 0.2783 - val_accuracy: 0.8854
Epoch 4/50
54/54 [==============================] - 144s 3s/step - loss: 0.3001 - accuracy: 0.8843 - val_loss: 0.2858 - val_accuracy: 0.8854
Epoch 5/50
54/54 [==============================] - 195s 4s/step - loss: 0.2682 - accuracy: 0.8848 - val_loss: 0.2082 - val_accuracy: 0.9115
Epoch 6/50
54/54 [==============================] - 173s 3s/step - loss: 0.2510 - accuracy: 0.8970 - val_loss: 0.3244 - val_accuracy: 0.8438
Epoch 7/50
54/54 [==============================] - 147s 3s/step - loss: 0.2124 - accuracy: 0.9242 - val_loss: 0.1652 - val_accuracy: 0.9115
Epoch 8/50
 9

In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss= history.history['val_loss']

In [ ]:
def predict(model,img):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array=tf.expand_dims(img_array,0)
    
    predictions=model.predict(img_array)
    
    predicted_class=class_names[numpy.argmax(predictions[0])]
    confidence=round(100*(numpy.max(predictions[0])),2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15,15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        predicted_class, confidence=predict(model,images[i].numpy())
    
        actual_class=class_names[labels[i]]
        plt.title(f"Atual:{actual_class}, \n Predicted:{predicted_class}.\n Confidence:{confidence}%")

In [ ]:
model_version=1
model.save(f"model/{model_version}")